In [1]:
import sys
sys.path.insert(0, '/kaggle/input/wheatdetection/wheat_detection')
import os

import torch
from torch.backends import cudnn

from backbone import EfficientDetBackbone
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from efficientdet.utils import BBoxTransform, ClipBoxes
from utils.utils import preprocess, invert_affine, postprocess

In [2]:
compound_coef = 0
imgs_dir = '../input/global-wheat-detection/test'
threshold = 0.3
iou_threshold = 0.2
input_size = 512
use_cuda = True

In [3]:
model = EfficientDetBackbone(compound_coef=compound_coef, num_classes=1,
                             ratios=[(1.0, 1.0), (1.4, 0.7), (0.7, 1.4)],
                             scales=[2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)])

if use_cuda:
    model.load_state_dict(torch.load('../input/wheatdetection/wheat_detection/pth/final_stage.pth', map_location=torch.device('cuda')))
else:
    model.load_state_dict(torch.load('../input/wheatdetection/wheat_detection/pth/final_stage.pth', map_location=torch.device('cpu')))
model.requires_grad_(False)
model.eval()
if use_cuda:
    model = model.cuda()

In [4]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))
    return " ".join(pred_strings)

In [5]:
results=[]
for img_dir in os.listdir(imgs_dir):
    if not img_dir.endswith('.jpg'):
        continue
    full_dir = os.path.join(imgs_dir, img_dir)
    ori_imgs, framed_imgs, framed_metas = preprocess(full_dir, max_size=input_size)
    if use_cuda:
        x = torch.stack([torch.from_numpy(fi).cuda() for fi in framed_imgs], 0)
    else:
        x = torch.stack([torch.from_numpy(fi) for fi in framed_imgs], 0)
    x = x.to(torch.float32).permute(0, 3, 1, 2)
    with torch.no_grad():
        features, regression, classification, anchors = model(x)

        regressBoxes = BBoxTransform()
        clipBoxes = ClipBoxes()
        out = postprocess(x,
                          anchors, regression, classification,
                          regressBoxes, clipBoxes,
                          threshold, iou_threshold)
    out = invert_affine(framed_metas, out)
    
    for i in range(len(ori_imgs)):
        if len(out[i]['rois']) == 0:
            result = {
                'image_id': img_dir.split('.')[0],
                'PredictionString': ''
            }
            results.append(result)
            continue
        boxes = out[i]['rois'].astype(np.int)
        scores = out[i]['scores'].astype(np.float16)

        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]

        result = {
            'image_id': img_dir.split('.')[0],
            'PredictionString': format_prediction_string(boxes, scores)
        }
        results.append(result)

In [6]:
test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
test_df.head()

,image_id,PredictionString
0,51b3e36ab,0.9868 496 359 319 123 0.9800 112 840 148 93 0...
1,51f1be19e,0.9258 770 884 146 104 0.8970 846 272 129 190 ...
2,cc3532ff6,0.9917 775 825 166 166 0.9771 377 1 84 96 0.96...
3,348a992bb,0.9868 733 216 140 87 0.9624 138 37 116 83 0.9...
4,53f253011,0.9722 306 60 91 126 0.9722 230 833 118 103 0....


In [7]:
test_df.to_csv(r'/kaggle/working/submission.csv', index=False)